In [331]:
#!pip install rdflib
#!pip install pydot2
#!pip install pydotplus

In [332]:
import rdflib
from rdflib.namespace import RDFS
from rdflib import URIRef, BNode, Literal
import re
from collections import defaultdict
import numpy as np
import pandas as pd
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
import matplotlib.pyplot as plt

In [333]:
def ns(url):
  url = url.replace("http://buildsys.org/ontologies/brick#", "")
  url = url.replace("http://buildsys.org/ontologies/brickFrame#", "")
  return url

BRICKF = rdflib.Namespace('http://buildsys.org/ontologies/brickFrame#')
BRICK  = rdflib.Namespace('http://buildsys.org/ontologies/brick#')
TAG    = rdflib.Namespace('http://buildsys.org/ontologies/brickTag#')
TAGSET = rdflib.Namespace('http://buildsys.org/ontologies/brickTagSet#')

g = rdflib.Graph()
g.bind('bf', BRICKF)
g.bind('tag', TAG)
g.bind('ts', TAGSET)
g.bind(':', BRICK)
result = g.parse('BrickFrame.ttl', format='n3')
print(len(g))

126


### Load Tag and TagSets from Definition

In [334]:
dfTags=pd.read_excel('Schema Engineering.xlsx',"Tags")
schemaTags=set(pd.unique(dfTags.Tag.dropna().ravel()))
len(schemaTags)
dfTags.head()

,Dimension,Tag,Definition,Related Terminologies,hasSynonym,Author,Reviewed,Review Count,Comments
0,Equipment,HVAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Equipment>Energy>Power,Load,NaN,Eletrical load,NaN,Jason,NaN,NaN,NaN
2,Equipment>Energy>Power,Bus,"A metallic strip or bar (typically copper, bra...",NaN,NaN,Jason,NaN,NaN,NaN
3,Equipment>Energy>Power,Battery,NaN,NaN,NaN,Jason,NaN,NaN,NaN
4,Equipment>Energy>Power,Solar,NaN,Solar panel,NaN,Jason,NaN,NaN,NaN


In [335]:
dfTagSets=pd.read_excel('Schema Engineering.xlsx',"TagSets")
schemaTagSets=set()
for ts in pd.unique(dfTagSets.TagSet.dropna().ravel()):
  schemaTagSets.add(ts.replace(' ','_'))
for ts in pd.unique(dfTagSets.hasSynonym.dropna().ravel()):
  for ts2 in ts.split(","):
   schemaTagSets.add(ts2.replace(' ','_'))
len(schemaTagSets)
dfTagSets.head()

,Dimension,TagSet,hasSynonym,Definition,hasUnit,hasAbbreviation,can be named? (can be instantiated?),can be unnamed? (cannot be instantiated?),Author,Reviewed,Review Count,Discussion
0,Equipment,Fire Control Panel,FCP,NaN,NaN,FCP,NaN,NaN,Bharath,NaN,NaN,NaN
1,Equipment>Exhaust,Exhaust Fan,NaN,NaN,NaN,NaN,NaN,NaN,Jason,NaN,NaN,NaN
2,Equipment>HVAC,Space Heater,NaN,NaN,NaN,NaN,O,X,Jason,NaN,NaN,NaN
3,Equipment>HVAC,Air Handler Unit,AHU,A device used to regulate and circulate air as...,NaN,AHU,O,X,Joern,NaN,NaN,NaN
4,Equipment>HVAC,Computer Room Air Conditioning,CRAC,NaN,NaN,CRAC,NaN,NaN,Bharath,NaN,NaN,NaN


In [336]:
schemaUsedTags=set()
schemaTagSetTags={}
for ts in schemaTagSets:
  schemaUsedTags.update(ts.split('_'))
  schemaTagSetTags[ts]=set(ts.split('_'))
schemaMissingTags=schemaUsedTags - schemaTags
print("Missing Tags:" + str(len(schemaMissingTags)))
print(schemaMissingTags)

Missing Tags:1
{''}


In [337]:
def IndivName(name):
  return re.sub(r'\s','_',re.sub(r'[^\d\w\s]', '', name))

### Write Tags

write header

In [338]:
nsTag   = ":"
fo = open('BrickTag.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix : <http://buildsys.org/ontologies/BrickTag#> .\n\n")
fo.write("<http://buildsys.org/ontologies/BrickTag>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write('\trdfs:comment "Domain Tag Definition"@en .\n\n')

44

Write Tag Hierachy

In [339]:
brickTags={}
for hir in pd.unique(dfTags.Dimension.dropna().ravel()):
  tags=hir.split('>')
  atags=""
  for i in range(len(tags)):
    tag=tags[i]
    otags=atags
    atags=(atags+"_"+tag).strip("_")
    if atags not in brickTags:
      indivLocName=nsTag+IndivName(atags)
      if i>0:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+brickTags[otags]+";")
      else:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   bf:Tag;")
      fo.write('\n\t\t\t bf:isHierarchical  "";')
      fo.write('\n\t\t\t rdfs:label "'+tag+'"@en .\n')
      brickTags[atags]=indivLocName;
      parent=tag;

Add tag leaves

In [340]:
# create location individuals
for idx in dfTags.index:
  #parent=brickTags[str(dfTags.loc[idx, "Dimension"]).split('>')[-1]]
  parent=brickTags[str(dfTags.loc[idx, "Dimension"]).replace('>','_')]
  indivLocName=nsTag + IndivName(str(dfTags.loc[idx, "Tag"]))
  fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+parent+";")
  fo.write('\n\t\t\t rdfs:label "'+str(dfTags.loc[idx, "Tag"])+'"@en .\n')
  brickTags[tag]=indivLocName;
  #print(parent,indivLocName)

In [341]:
fo.close()

### Write TagSets

write headers

In [342]:
nsTag= "tag:" # "ts:"
nsTagSet= ":" # "ts:"
fo = open('Brick.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix tag: <http://buildsys.org/ontologies/BrickTag#> .\n")
fo.write("@prefix :    <http://buildsys.org/ontologies/Brick#> .\n\n")
fo.write("<http://buildsys.org/ontologies/Brick>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickTag> ;\n")
fo.write('\trdfs:comment "Domain TagSet Definition"@en .\n\n')

47

Write TagSet Hierachy

In [343]:
brickTagSets={}
for hir in pd.unique(dfTagSets.Dimension.dropna().ravel()):
  tags=hir.split('>')
  atags=""
  for i in range(len(tags)):
    tag=tags[i]
    otags=atags
    atags=(atags+"_"+tag).strip('_')
    if atags not in brickTagSets:
      indivLocName=nsTagSet + IndivName(atags)
      if i>0:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+brickTagSets[otags]+";")
      else:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   bf:TagSet;")
      fo.write('\n\t\t\t bf:isHierarchical  "";')
      fo.write('\n\t\t\t rdfs:label "'+tag+'"@en .\n')
      brickTagSets[atags]=indivLocName;

Add TagSets leaves

In [344]:
# create location individuals
for idx in dfTagSets.index:
  tagsets=set([str(dfTagSets.loc[idx, "TagSet"])]) | set(str(dfTagSets.loc[idx, "hasSynonym"]).split(","))
  ots=None
  for tagset in tagsets:
    if tagset!="nan":
        #tagset=str(dfTagSets.loc[idx, "TagSet"])
      #parent= brickTagSets[str(dfTagSets.loc[idx, "Dimension"]).split('>')[-1]]
      parent= brickTagSets[str(dfTagSets.loc[idx, "Dimension"]).replace('>',"_")]
      indivLocName=nsTagSet + IndivName(tagset)
      fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+parent+";")
      for tag in tagset.split():
         #fo.write('\n\t\t\t bf:hasTag  tag:'+tag+';')
         fo.write('\n\t\t\t rdfs:subClassOf [ a owl:Restriction ; owl:onProperty :hasTag ; owl:someValuesFrom tag:'+tag+' . ];')
      if ots:
        fo.write('\n\t\t\t owl:equivalentClass '+ots+' ;')
      fo.write('\n\t\t\t rdfs:label "'+str(dfTagSets.loc[idx, "TagSet"])+'"@en .\n')
      brickTagSets[tagset]=indivLocName;
      ots=indivLocName;
      #print(parent,indivLocName)

In [345]:
fo.close()

### New Structure

Classify Tags by Type

In [346]:
dfMeasTags=dfTags.loc[dfTags.Dimension == "MeasurementProperty>PhysicalProperties"]
dfLocTags=dfTags.loc[dfTags.Dimension == "Location"]
dfPointTags=dfTags.loc[dfTags.Dimension == "Point"]
dfEquipTags=dfTags.loc[dfTags.Dimension.str.startswith("Equipment")]

def getPointDim(istr):
  for pt in dfPointTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedPoint"

def getMeasDim(istr):
  for pt in dfMeasTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedMeasurement"

def getLocDim(istr):
  for pt in dfLocTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedLocation"

def getEquipDim(istr):
  for pt in dfEquipTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedEquipment"

def getLastDim(istr):
  if ">"   not in istr:
    return "UndefinedEquipment"
  else:
    sstr=istr.split(">")
    return IndivName(sstr[-1])  

In [347]:
dfTagSets['PointDim']=dfTagSets.TagSet.apply(getPointDim)
dfTagSets['MeasurementDim']=dfTagSets.TagSet.apply(getMeasDim)
dfTagSets['LocationDim']=dfTagSets.TagSet.apply(getLocDim)
#dfTagSets['EquipmentDim']=dfTagSets.TagSet.apply(getEquipDim)
dfTagSets['EquipmentDim']=dfTagSets.Dimension.apply(getLastDim)
dfPointTagSets=dfTagSets.loc[dfTagSets.Dimension.str.startswith("Point")]

In [348]:
def findMin(S, R):
  maxCost = 0
  minElement = None
  for i in S:
    try:
      cost = len(S[i].intersection(R))
      if cost > maxCost:
        maxCost = cost
        minElement = i
    except:
      # Division by zero, ignore
      pass
  return minElement


In [349]:
tagsets={}
tagsetsSup={}
equivTagSets=[]
for dim in ['PointDim', 'MeasurementDim', 'LocationDim', 'EquipmentDim']:
  dimT=dim.replace('Dim', '')
  tagsetsSup[dimT]=['TagSet']
  tagsets[dimT]=set()
  for dimL in pd.unique(dfPointTagSets[dim].dropna().ravel()):
    tagsetsSup[dimL]=[dimT]
    #if not dimL.startswith("Undefined"):
    tagsets[dimL]=set(dimL.split('_'))
for idx in dfPointTagSets.index:
  tss=set([str(dfTagSets.loc[idx, "TagSet"])]) | set(str(dfTagSets.loc[idx, "hasSynonym"]).split(","))
  tss2=set()
  if "nan" in tss: tss.remove("nan")
  for tagset in tss:
    if tagset!="nan":
      tagset=IndivName(tagset)
      tss2.add(tagset)
      tagsets[tagset] = set(tagset.split('_'))
      equip=dfTagSets.loc[idx, 'EquipmentDim']
      if equip!="nan" and equip != "UndefinedEquipment" and equip not in tagsets[tagset]:
        for tag in equip.split('_'):
          tagsets[tagset].add(tag)
      parents=[]
      for dim in ['PointDim', 'MeasurementDim', 'LocationDim', 'EquipmentDim']:
        parents.append(dfTagSets.loc[idx, dim])
      tagsetsSup[tagset]=parents
  if len(tss2)>1:
    equivTagSets.append(tss2)

In [350]:
tagsetsSupAll={}
for tsA in tagsetsSup:
  if tsA not in tagsetsSupAll: tagsetsSupAll[tsA]=set()
  for tsB in tagsetsSup[tsA]:
    tagsetsSupAll[tsA].add(tsB)
c=0
for tsA in tagsetsSupAll:
  for tsB in tagsetsSupAll[tsA]:
    c+=1
c


1910

In [351]:
i=0
missingRel={}
for tsA in tagsets:
  i=i+1
  j=0
  for tsB in tagsets:
    j=j+1
    if tsA==tsB: #j <= i or 
      pass
    if tagsets[tsA] > tagsets[tsB]: # tsB parentOf tsA
      if tsA not in tagsetsSup:
        if tsA not in missingRel: missingRel[tsA]=set()
        missingRel[tsA].add(tsB)
      else:
        anySup=False
        for tsC in tagsetsSup[tsA]:
          if tsC in tagsets and tagsets[tsC] > tagsets[tsB]: # tsB parentOf tsC
            anySup = True
        if not anySup:
          if tsA not in missingRel: missingRel[tsA]=set()
          missingRel[tsA].add(tsB)
c=0
for tsA in missingRel:
  for tsB in missingRel[tsA]:
    c+=1
c

1913

In [352]:
# remove redundant entries
ignoreTagSets=['Point','Measurement','Location','Equipment',"UndefinedPoint","UndefinedEquipment","UndefinedLocation","UndefinedMeasurement"]
remTsA=set(ignoreTagSets)
for tsA in missingRel:
  remTsB=set(ignoreTagSets)
  for tsB in missingRel[tsA]:
    for tsC in missingRel[tsA]:
      if tagsets[tsC] > tagsets[tsB]:
        remTsB.add(tsB)
  missingRel[tsA]=missingRel[tsA].difference(remTsB)
  if len(missingRel[tsA])==0: remTsA.add(tsA)
for tsA in remTsA:
  if tsA in missingRel:
    del missingRel[tsA]

for tsA in missingRel:
  if tsA not in tagsetsSupAll: tagsetsSupAll[tsA]=set()
  for tsB in missingRel[tsA]:
    tagsetsSupAll[tsA].add(tsB)
    
c=0
for tsA in missingRel:
  for tsB in missingRel[tsA]:
    c+=1
c

1119

In [353]:
# compute minimal set cover of parents
redundantSup={}
ignoreTagSets=['Point','Measurement','Location','Equipment',"UndefinedPoint","UndefinedEquipment","UndefinedLocation","UndefinedMeasurement"]
remTsA=set(ignoreTagSets)
tagsetsSupMin={}
for tsA in tagsetsSupAll:
  if tsA not in tagsets:
    tagsetsSupMin=tagsetsSupAll[tsA]
    print(tsA+" not in tagsets")
    continue
  unassigned=set(tagsets[tsA])
  S={}
  minCov=set()
  for tsB in tagsetsSupAll[tsA]: 
    if tsB!=tsA and tsB in tagsets and len(tagsets[tsB])>0:
      S[tsB]=set(tagsets[tsB])
  while len(unassigned)>0 and len(S)>0:
    tsB=findMin(S, unassigned)
    if tsB is None: break
    unassigned=unassigned.difference(S[tsB])
    minCov.add(tsB)
    del S[tsB]
  for tag in unassigned:
    if tag!=tsA:
      minCov.add(tag)
  if minCov:
    tagsetsSupMin[tsA]=minCov
    #print(tsA+": "+str(minCov))
c=0
for tsA in tagsetsSupMin:
  for tsB in tagsetsSupMin[tsA]:
    c+=1
c

1925

In [354]:
# identify redundant parents exeding minimal set cover
redundantSup={}
for tsA in tagsetsSupAll:
  if tsA in tagsetsSupMin:
    redundantSup[tsA]=tagsetsSupAll[tsA].difference(tagsetsSupMin[tsA])
c=0
for tsA in redundantSup:
  for tsB in redundantSup[tsA]:
    c+=1
c

1021

In [355]:
for tsA in redundantSup:
  for tsB in redundantSup[tsA]:
    if tsA in tagsetsSupAll and tsB in tagsetsSupAll[tsA]:
      if not tsB.startswith("Undefined") and tsB!="":  
        tagsetsSupAll[tsA].remove(tsB)
    #g.remove((tagsetsNode[tsA],URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),tagsetsNode[tsB]))
for tsA in tagsetsSupMin:
  for tsB in tagsetsSupMin[tsA]:
    pass
    #if tsA not in tagsetsSupAll: tagsetsSupAll[tsA]=set([tsB])
    #else: tagsetsSupAll[tsA].add(tsB)
    #g.add((tagsetsNode[tsA],URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),tagsetsNode[tsB]))

c=0
for tsA in tagsetsSupAll:
  for tsB in tagsetsSupAll[tsA]:
    c+=1
c

1742

In [356]:
c=0
for tsA in tagsets: c+=len(tagsets[tsA])
c

2644

In [357]:
for tsA in tagsetsSupAll:
  if tsA in tagsets:
    uniqueTags=set(tagsets[tsA])
    if tsA in tagsetsSupAll:
      for tsB in tagsetsSupAll[tsA]:
        if tsB in tagsets:
          commontags=uniqueTags.intersection(tagsets[tsB])
          for tag in commontags:
            if tag in tagsets[tsA]:
              #g.remove((tagsetsNode[tsA],None,tagsetsTagsNode[tsA][tag]))
              tagsets[tsA].remove(tag)
c=0
for tsA in tagsets: c+=len(tagsets[tsA])
c

1148

In [358]:
nsTag= "tag:" # "ts:"
nsTagSet= ":" # "ts:"
fo = open('BrickV2.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix tag: <http://buildsys.org/ontologies/BrickTag#> .\n")
fo.write("@prefix :    <http://buildsys.org/ontologies/Brick#> .\n\n")
fo.write("<http://buildsys.org/ontologies/Brick>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickTag> ;\n")
fo.write('\trdfs:comment "Domain TagSet Definition"@en .\n\n')

47

In [359]:
brickTagSets={}
for dim in ['PointDim','MeasurementDim','LocationDim','EquipmentDim']:
  dimT=dim.replace('Dim','')
  fo.write("\n :"+dimT+"  rdfs:subClassOf   bf:TagSet;")
  fo.write('\n\t\t\t bf:isHierarchical  "";')
  fo.write('\n\t\t\t rdfs:label "'+dimT+' Dimension"@en .\n')
  for dimL in pd.unique(dfPointTagSets[dim].dropna().ravel()):
    fo.write("\n :"+dimL+"  rdfs:subClassOf   :"+dimT+";")
    fo.write('\n\t\t\t bf:isHierarchical  "";')
    fo.write('\n\t\t\t rdfs:label "'+dimL+'"@en .\n')
    brickTagSets[dimL]=dimL;

In [360]:
for ts in tagsets:
  supClass=""
  if ts in tagsetsSupAll:
    for ts2 in tagsetsSupAll[ts]:
      if ts2=="TagSet": supClass+=", bf:"+ts2;
      else: supClass+=", :"+ts2;
  if supClass=="":
    supClass=":Equipment"
  fo.write("\n :"+ts+"  rdfs:subClassOf   "+supClass.strip(',').strip()+";")
  #fo.write('\n\t\t\t bf:isHierarchical  "";')
  for tag in tagsets[ts]:
    if not tag.startswith("Undefined") and tag!="":
     fo.write('\n\t\t\t rdfs:subClassOf [ a owl:Restriction ; owl:onProperty :hasTag ; owl:someValuesFrom tag:'+tag+' . ];')
  fo.write('\n\t\t\t rdfs:label "'+ts.replace('_',' ')+'"@en .\n')

In [361]:
for eqTS in equivTagSets:
  ots=None
  for ts in eqTS:
    if ots:
      #fo.write('\n :'+ts+'   owl:equivalentClass  :'+ots+' .')
      fo.write('\n :'+ts+'   bf:similarClass    :'+ots+' .')
    ots=ts;   

In [362]:
fo.close()

In [363]:
brickTagSets={}
for dim in ['PointDim','MeasurementDim','LocationDim','EquipmentDim']:
  dimT=dim.replace('Dim','')
  fo.write("\n :"+dimT+"  rdfs:subClassOf   bf:TagSet;")
  fo.write('\n\t\t\t bf:isHierarchical  "";')
  fo.write('\n\t\t\t rdfs:label "'+dimT+' Dimension"@en .\n')
  for dimL in pd.unique(dfPointTagSets[dim].dropna().ravel()):
    fo.write("\n :"+dimL+"  rdfs:subClassOf   :"+dimT+";")
    fo.write('\n\t\t\t bf:isHierarchical  "";')
    fo.write('\n\t\t\t rdfs:label "'+dimL+'"@en .\n')
    brickTagSets[dimL]=dimL;

ValueError: I/O operation on closed file.

In [ ]:
# create location individuals
for idx in dfTagSets.index:
  tagsets=set([str(dfTagSets.loc[idx, "TagSet"])]) | set(str(dfTagSets.loc[idx, "hasSynonym"]).split(","))
  ots=None
  for tagset in tagsets:
    if tagset!="nan":
      indivLocName=nsTagSet + IndivName(tagset)
      parent=""
      for dim in ['PointDim','MeasurementDim','LocationDim','EquipmentDim']:
        parent+=", :"+dfTagSets.loc[idx, dim]
      fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+parent.strip(',')+";")
      for tag in tagset.split():
         fo.write('\n\t\t\t rdfs:subClassOf [ a owl:Restriction ; owl:onProperty :hasTag ; owl:someValuesFrom tag:'+tag+' . ];')
      if ots:
        fo.write('\n\t\t\t owl:equivalentClass '+ots+' ;')
      fo.write('\n\t\t\t rdfs:label "'+str(dfTagSets.loc[idx, "TagSet"])+'"@en .\n')
      brickTagSets[tagset]=indivLocName;
      ots=indivLocName;
      #print(parent,indivLocName)

In [ ]:
fo.close()